In [56]:
!pip install torchinfo


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [57]:
import torch
import sys
import os
from torchinfo import summary

# Ajouter le chemin pour importer ResNet18
sys.path.append(os.path.abspath("../LAB1"))
from resnet import ResNet18

def calculate_score(structured_pruning=0.2, global_pruning=0.2, is_half_precision=True):
    # Créer le modèle
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = ResNet18().to(device)
    #checkpoint = torch.load("./pruned_model.pth", map_location=device)
    #model.load_state_dict(checkpoint, strict=False) 
    print("Modèle chargé en half precision !")
    
    # Afficher les statistiques du modèle
    stats = summary(model, (1, 3, 32, 32), verbose=0)
    
    total_params = stats.total_params
    total_macs = stats.total_mult_adds  # Nombre d'opérations MAC (approximation des FLOPs)
    
    # Convertir MAC en FLOPs (environ 2 FLOPs par MAC)
    total_flops = total_macs * 2
    
    # Calculer le score
    p_s = structured_pruning
    p_u = global_pruning
    q_w = 16 if is_half_precision else 32  # 16 bits pour half precision, 32 pour full
    
    # On suppose q_a = 32 bits pour les activations
    max_q = max(q_w, 16)
    
    w = total_params
    f = total_flops
    
    param_term = (1 - (p_s + p_u)) * (q_w/32) * w / (5.6e6)
    ops_term = (1 - p_s) * (max_q/32) * f / (8.3e8)
    
    score = param_term + ops_term
    
    print(f"Nombre total de paramètres: {total_params:,}")
    print(f"Nombre total d'opérations (FLOPs): {total_flops:,}")
    print(f"Score calculé: {score:.4f}")
    
    return score

# Calculer le score avec vos paramètres
score = calculate_score(structured_pruning=0.2, global_pruning=0.2, is_half_precision=True)

Modèle chargé en half precision !
Nombre total de paramètres: 11,173,962
Nombre total d'opérations (FLOPs): 1,110,864,660
Score calculé: 1.1340


In [62]:
score = calculate_score(structured_pruning=0.0, global_pruning=0.7, is_half_precision=True)


Nombre total de paramètres: 11,173,962
Nombre total d'opérations (FLOPs): 1,110,864,660
Score calculé: 0.9685


In [63]:
score = calculate_score(structured_pruning=0.3, global_pruning=0.2, is_half_precision=True)

Nombre total de paramètres: 11,173,962
Nombre total d'opérations (FLOPs): 1,110,864,660
Score calculé: 0.9673


In [64]:
score = calculate_score(structured_pruning=0.5, global_pruning=0.5, is_half_precision=True) 

Nombre total de paramètres: 11,173,962
Nombre total d'opérations (FLOPs): 1,110,864,660
Score calculé: 0.3346


In [ ]:
score

Modele de base

In [60]:
import torch
import sys
import os
from torchinfo import summary

# Ajouter le chemin pour importer ResNet18
sys.path.append(os.path.abspath("../LAB1"))
from resnet import ResNet18

def calculate_score(structured_pruning=0.2, global_pruning=0.2, is_half_precision=True):
    # Créer le modèle
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = ResNet18().to(device)
    
    # Afficher les statistiques du modèle
    stats = summary(model, (1, 3, 32, 32), verbose=0)
    
    total_params = stats.total_params
    total_macs = stats.total_mult_adds  # Nombre d'opérations MAC (approximation des FLOPs)
    
    # Convertir MAC en FLOPs (environ 2 FLOPs par MAC)
    total_flops = total_macs * 2
    
    # Calculer le score
    p_s = structured_pruning
    p_u = global_pruning
    q_w = 16 if is_half_precision else 32  # 16 bits pour half precision, 32 pour full
    
    # On suppose q_a = 32 bits pour les activations
    max_q = max(q_w, 16 if is_half_precision else 32 )
    
    w = total_params
    f = total_flops
    
    param_term = (1 - (p_s + p_u)) * (q_w/32) * w / (5.6e6)
    ops_term = (1 - p_s) * (max_q/32) * f / (8.3e8)
    
    score = param_term + ops_term
    
    print(f"Nombre total de paramètres: {total_params:,}")
    print(f"Nombre total d'opérations (FLOPs): {total_flops:,}")
    print(f"Score calculé: {score:.4f}")
    
    return score

# Calculer le score avec vos paramètres
score = calculate_score(structured_pruning=0, global_pruning=0, is_half_precision=False)

Nombre total de paramètres: 11,173,962
Nombre total d'opérations (FLOPs): 1,110,864,660
Score calculé: 3.3337
